In [6]:
import torch
from torchvision import transforms, datasets
from torch.utils.data import DataLoader
from torch.utils.data import random_split

# Define your transformations
data_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

''' for fruits-360 dataset with preslit data
# Load your dataset
train_dataset = datasets.ImageFolder(root='fruits-360/Training', transform=data_transforms)
val_dataset = datasets.ImageFolder(root='fruits-360/Test', transform=data_transforms)

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)'''

# for custom dataset
# Load the full dataset
train_dataset = datasets.ImageFolder(root='RealWaste', transform=data_transforms)

# Split the dataset
train_size = int(0.8 * len(train_dataset))  # 80% for training
test_size = len(train_dataset) - train_size  # 20% for testing
train_dataset, eval_dataset = random_split(train_dataset, [train_size, test_size])

# Create data loaders
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
eval_loader = DataLoader(eval_dataset, batch_size=32, shuffle=False)

# Get the number of classes
num_classes = len(train_dataset.dataset.classes)

# Print the number of classes
print('Number of classes:', num_classes)


Number of classes: 9


In [7]:
import torchvision.models as models

model = models.efficientnet_b0(pretrained=True)  # Assuming this is available

# Modify the classifier for your dataset
num_ftrs = model.classifier[1].in_features
model.classifier[1] = torch.nn.Linear(num_ftrs, num_classes)  # 'num_classes' is the number of your dataset classes


/Users/vickieduong/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/Users/vickieduong/anaconda3/lib/python3.11/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [8]:
import torch.optim as optim

criterion = torch.nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

device = torch.device("cpu")
if torch.cuda.is_available():
	device = torch.device("cuda")
elif torch.backends.mps.is_available():
	device = torch.device("mps")
else:
	device = torch.device("cpu")
print(device)
	

mps


In [9]:
from tqdm import tqdm

num_epochs = 10
model.to(device)

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    # Wrap your data loader with tqdm for a progress bar
    train_loader = tqdm(train_loader)

    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs)
        loss = criterion(outputs, labels)

        # Backward and optimize
        loss.backward()
        optimizer.step()

        running_loss += loss.item()
        
        # Update the progress bar with the loss information
        train_loader.set_description(f'Epoch {epoch+1}/{num_epochs}')
        train_loader.set_postfix(loss=loss.item())
    
    # Optionally, you can add a validation loop here and use tqdm in the same way
	# to monitor the validation loss and accuracy
with torch.no_grad():
    model.eval()
    val_loss = 0.0
    val_loader = tqdm(eval_loader)
    for inputs, labels in val_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = model(inputs)
        loss = criterion(outputs, labels)
        val_loss += loss.item()

        val_loader.set_description(f'Validation Epoch {epoch+1}/{num_epochs}')
        val_loader.set_postfix(loss=loss.item())
torch.save(model.state_dict(), 'model_weights.pth')



Epoch 3/10:  56%|█████▋    | 67/119 [04:53<03:48,  4.39s/it, loss=0.479] 


KeyboardInterrupt: 